# Simple Substitution
`w266 Final Project: Crosslingual Word Embeddings`

The code in this notebook was used to develop an algorithm to generate crosslingual word embeddings by training on a monolingual corpus and substituting translations at runtime.

# Notebook Setup

In [1]:
# general imports
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# tell matplotlib not to open a new window
%matplotlib inline

# autoreload modules
%load_ext autoreload
%autoreload 2

In [9]:
# filepaths
BASE = '/Users/mmillervedam/Documents/MIDS/w266' #'/home/mmillervedam/' 
PROJ = '/Users/mmillervedam/Documents/MIDS/w266/FinalProject'#'/home/mmillervedam/ProjectRepo'
FPATH_EN = BASE + '/Data/test/wiki_en_10K.txt' # first 10000 lines from wiki dump
FPATH_ES = BASE + '/Data/test/wiki_es_10K.txt' # first 10000 lines from wiki dump
#FULL_EN = BASE + '/Data/en/full.txt'
#FULL_ES = BASE + '/Data/es/full.txt'
EN_ES_DICT = PROJ +'/XlingualEmb/data/dicts/en.es.panlex.all.processed'
EN_IT_DICT  = PROJ +'/XlingualEmb/data/dicts/en.it.panlex.all.processed'
EN_IT_RAW = PROJ + '/XlingualEmb/data/mono/en_it.shuf.10k'
EN_IT_RAW = PROJ + '/XlingualEmb/data/mono/en_it.shuf.10k'

# Load & Preprocess Data
__`ORIGINAL AUTHORS SAY:`__ "Normally, the monolingual word embeddings are trained on billions of words. However, getting that much of monolingual data for a low-resource language is also challenging. That is why we only select the top 5 million sentences (around 100 million words) for each language." - _Section 5.1, Duong et. al._ 

In [3]:
from parsing import Corpus, Vocabulary, batch_generator

### Corpus

In [11]:
# load corpus
en_it_data = Corpus(EN_IT_RAW)

In [12]:
# Corpus Stats
!wc {EN_IT_RAW}

   20000  430928 3746786 /Users/mmillervedam/Documents/MIDS/w266/FinalProject/XlingualEmb/data/mono/en_it.shuf.10k


__`i.e.:`__ 20K sentences (10K in each language) with ~430K tokens
> So this must not be their full data For now, I'm just going to look at the top 20K words and see what happens. In reality we should probably modify the Vocab class so that it explicily collects the top words for each language separately and then concatenates the index.

### Dictionary

In [13]:
# loading english-italian dictionary
pld = pd.read_csv(EN_IT_DICT, sep='\t', names = ['en', 'it'], dtype=str)
en_set = set(pld.en.unique())
it_set = set(pld.it.unique())

In [93]:
# dictionary vocab lengths:
print('EN:', len(en_set))
print('IT:', len(it_set))

EN: 266450
IT: 258641


In [127]:
# Create dictionary for ease of runtime translation
# WARNING this takes a sec to run
bi_dict = pld.groupby(['en'])['it'].unique().to_dict()

In [129]:
# add other direction
# WARNING this takes another sec to run
bi_dict.update(pld.groupby(['it'])['en'].unique().to_dict())

In [130]:
# demo en to it
bi_dict['en_go'][:5]

array(['it_aggirare', 'it_andai', 'it_andara', 'it_andare',
       'it_andare_avanti'], dtype=object)

In [134]:
# demo it to en
bi_dict['it_ciao'][:5]

array(['en_adieu', 'en_bye-bye', 'en_bye', 'en_cheerio', 'en_ciao'], dtype=object)

### Vocabulary

In [16]:
# train multilingual Vocabulary
en_it_vocab = Vocabulary(en_it_data.gen_tokens(), size = 100000)

### CBOW Data Generator
__`CHECK PAPER for HYPERPARAMS!`__: I can't seem to find where they talk abou the context window size, embedding size and batch size they use -- it may actually be in the Vulic and Moens paper instead of the Duong one.

__`RLH Update`__: Duong et al. section 6, footnote 4: "Default learning rate of 0.025, negative sampling with 25 samples, subsampling rate of value 1e−4, embedding dimension d = 200, window size cs = 48 and run for 15 epochs"


In [24]:
BATCH_SIZE = 48
WINDOW_SIZE = 1
MAX_EPOCHS = 1 # fail safe

In [25]:
batched_data = batch_generator(en_it_data, 
                               en_it_vocab, 
                               BATCH_SIZE, 
                               WINDOW_SIZE, 
                               MAX_EPOCHS)

In [26]:
# sanity check
for context, label in batched_data:
    print("CONTEXT IDS:", context[:5])
    print("LABEL IDS:", label[:5])
    break

CONTEXT IDS: [[0, 1], [0, 1], [0, 34], [20, 17318], [34, 1638]]
LABEL IDS: [25668, 37957, 20, 34, 17318]


# Base Model - no word sub yet!
__`CODE NOTES:`__ To get this running I had to hard code the context length (set to 2) inside `BuildCoreGraph()` where we generate `self.input_` in line 102. That should really be inferred from the `self.context_` itself but it doesn't seem to like the placeholder dimension (we don't have a span length until runtime). Does tensorflow not have a vectorized average? Something to fix (later). I also had to hard code the number of samples for softmax (I had originally put this as a `tf.placeholder_with_default` thinking we could pass it in to the training function (since its a training parameter) but TF kicked out an error message asking for an integer so for now I'll just give it what it wants. I need to think more about why TF doesn't want this changing from batch to batch. (or if there is another reason it wants an int).

### Initialize the model

In [83]:
from models import BiW2V

EMBEDDING_SIZE = 128

# create model
model = BiW2V(index = en_it_vocab.index, H = EMBEDDING_SIZE)

# intialize TF graphs
model.BuildCoreGraph()
model.BuildTrainingGraph()
model.BuildValidationGraph()

### Training

__`IMPORTANT!`__ right now the model only works with a window of 1 because the feed dict can't handle context windows of different lengths. We'll either need to figure out how to have a variable length dimension or else add extra padding to the sentences to account for the window size.

In [147]:
model.train(200, batched_data)

... Model Initialized
	 <tf.Variable 'Embedding_Layer/ContextEmbeddings:0' shape=(48579, 128) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/WordEmbeddings:0' shape=(48579, 128) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/b:0' shape=(48579,) dtype=float32_ref>
... Starting Training
Average loss at step  0 :  0.308523583412
   Nearest to _en_the: _it_discrasia, _it_residenziali, _it_bulgharo, _en_gunboats, _it_mathematici, _en_william, _en_fifty-fifty, _it_3a,
   Nearest to _en_,: _en_roller, _en_ph., _it_1653, _en_anti-elitist, _it_incorrere, _en_too, _en_murdering, _it_macrofagi,
   Nearest to _en_.: _it_bavier, _en_naming, _en_spore, _it_rodigina, _it_all'accademia, _it_caselle, _it_passatempi, _en_evil,
   Nearest to _en_of: _en_brasfield, _it_automatica, _en_briton, _it_blücher, _it_bjarsmyr, _en_adulthood, _en_[[620]], _en_rulers,
   Nearest to _it_,: _it_juni, _en_leaning, _en_cliff, _it_registri, _it_torrette, _it_restaurati, _en_adulation, _en_precipitation,
Average loss a

In [86]:
# take a look at the embeddings
model.context_embeddings

array([[  5.65571594e-04,   6.43051637e-04,  -6.63136656e-04, ...,
         -4.45842772e-04,  -4.53017099e-04,  -6.84309460e-04],
       [  7.37371389e-04,  -2.08008365e-04,  -3.65211483e-04, ...,
         -7.20778073e-04,   8.04547279e-04,   5.35670901e-04],
       [  4.00696008e-04,  -1.33344976e-04,   1.50945591e-04, ...,
         -7.95033484e-05,   1.26949293e-04,   1.82113115e-04],
       ..., 
       [  4.10380628e-04,   3.62841412e-04,  -2.36171254e-04, ...,
         -1.01725709e-04,   2.44566298e-04,   5.58765954e-04],
       [  1.00196783e-04,  -6.64791907e-04,  -2.75951839e-04, ...,
          1.84948352e-04,   5.81149478e-04,  -2.65893847e-04],
       [ -9.84165599e-05,  -4.72841930e-04,   1.04621409e-04, ...,
          4.17812756e-04,  -5.55869134e-04,  -5.13212231e-04]], dtype=float32)

__`Hmmmm...`__ These don't look normalized to me. Something to return to?

# Model with Random Translation

### Initialize

In [152]:
from models import BiW2V_random

EMBEDDING_SIZE = 128

# create model
model1 = BiW2V_random(('en', 'it'), bi_dict, en_it_vocab.to_ids,
                      index = en_it_vocab.index, 
                      H = EMBEDDING_SIZE)

# intialize TF graphs
model1.BuildCoreGraph()
model1.BuildTrainingGraph()
model1.BuildValidationGraph()

... TF graph created for BiW2V model.
... TF graph created for BiW2V training.
... TF graph created for BiW2V validation.


### Train

In [153]:
model1.train(1, batched_data)

... Model Initialized
	 <tf.Variable 'Embedding_Layer/ContextEmbeddings:0' shape=(48579, 128) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/WordEmbeddings:0' shape=(48579, 128) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/b:0' shape=(48579,) dtype=float32_ref>
... Starting Training


AttributeError: 'list' object has no attribute 'squeeze'